In [1]:
%matplotlib inline
import os
import random

In [2]:
import numpy as np
import json
import matplotlib.pyplot

In [3]:
import pickle
from matplotlib.pyplot import imshow
from PIL import Image
from sklearn.manifold import TSNE
print("installed")

installed


In [5]:
images, pca_features, pca = pickle.load(open('pcafeatures.p', 'rb'))

for img, f in list(zip(images, pca_features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(img, f[0], f[1], f[2], f[3]))

image: ../../Dataset/img_align_celeba/177826.jpg, features: 35.04,-3.74,1.01,8.48... 
image: ../../Dataset/img_align_celeba/000997.jpg, features: -0.39,6.56,-23.71,-0.45... 
image: ../../Dataset/img_align_celeba/163448.jpg, features: 53.41,-11.88,18.07,-7.25... 
image: ../../Dataset/img_align_celeba/099166.jpg, features: -17.65,-19.86,-0.53,-16.97... 
image: ../../Dataset/img_align_celeba/199881.jpg, features: 5.69,-8.29,-29.23,-6.34... 


In our dataset that we've loaded, there are 9144 images. Although in principle, t-SNE works with any number of images, it's difficult to place that many tiles in a single image. So instead, we will take a random subset of 1000 images and plot those on a t-SNE instead. This step is optional, or you can try changing `num_images_to_plot`.

In [6]:
num_images_to_plot = 1000

if len(images) > num_images_to_plot:
    sort_order = sorted(random.sample(range(len(images)), num_images_to_plot))
    print(sort_order[-1])
    images = [images[i] for i in sort_order]
    
    pca_features = [pca_features[i] for i in sort_order]

1335


It is usually a good idea to first run the vectors through a faster dimensionality reduction technique like [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) to project your data into an intermediate lower-dimensional space before using t-SNE. This improves accuracy, and cuts down on runtime since PCA is more efficient than t-SNE. Since we have already projected our data down with PCA in the previous notebook, we can proceed straight to running the t-SNE on the feature vectors. Run the command in the following cell, taking note of the arguments:

- `n_components` is the number of dimensions to project down to. In principle it can be anything, but in practice t-SNE is almost always used to project to 2 or 3 dimensions for visualization purposes.
- `learning_rate` is the step size for iterations. You usually won't need to adjust this much, but your results may vary slightly. 
- `perplexity` refers to the number of independent clusters or zones t-SNE will attempt to fit points around. Again, it is relatively robust to large changes, and usually 20-50 works best. 
- `angle` controls the speed vs accuracy tradeoff. Lower angle means better accuracy but slower, although in practice, there is usually little improvement below a certain threshold.

In [7]:
X = np.array(pca_features)
tsne = TSNE(n_components=2, learning_rate=150, perplexity=30, angle=0.2, verbose=2).fit_transform(X)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.002s...
[t-SNE] Computed neighbors for 1000 samples in 0.093s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 10.778197
[t-SNE] Computed conditional probabilities in 0.039s
[t-SNE] Iteration 50: error = 69.4761810, gradient norm = 0.2759373 (50 iterations in 0.834s)
[t-SNE] Iteration 100: error = 68.9138565, gradient norm = 0.2606675 (50 iterations in 0.677s)
[t-SNE] Iteration 150: error = 69.9503784, gradient norm = 0.2535008 (50 iterations in 0.741s)
[t-SNE] Iteration 200: error = 69.1971130, gradient norm = 0.2684365 (50 iterations in 0.700s)
[t-SNE] Iteration 250: error = 68.6743469, gradient norm = 0.2660004 (50 iterations in 0.712s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 68.674347
[t-SNE] Iteration 300: error = 1.6901678, gradient norm = 0.0016543 (50 iterations in 0.648s)
[t-SNE] Iteration 350: error = 1.5941961, gradient norm = 0.0004

Internally, t-SNE uses an iterative approach, making small (or sometimes large) adjustments to the points. By default, t-SNE will go a maximum of 1000 iterations, but in practice, it often terminates early because it has found a locally optimal (good enough) embedding.

The variable `tsne` contains an array of unnormalized 2d points, corresponding to the embedding. In the next cell, we normalize the embedding so that lies entirely in the range (0,1).

In [8]:
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

Finally, we will compose a new RGB image where the set of images have been drawn according to the t-SNE results. Adjust `width` and `height` to set the size in pixels of the full image, and set `max_dim` to the pixel size (on the largest size) to scale images to.

In [9]:
width = 4000
height = 3000
max_dim = 100

full_image = Image.new('RGBA', (width, height))
for img, x, y in zip(images, tx, ty):
    tile = Image.open(img)
    rs = max(1, tile.width/max_dim, tile.height/max_dim)
    tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))

matplotlib.pyplot.figure(figsize = (16,12))
imshow(full_image)

FileNotFoundError: [Errno 2] No such file or directory: '../../Dataset/img_align_celeba/000997.jpg'

You can save the image to disk:

In [10]:
full_image.save("example-tSNE_1.png")

Now that we have generated our t-SNE, one more nice thing we can optionally do is to take the 2d embedding and assign it to a grid, using [RasterFairy](https://github.com/Quasimondo/RasterFairy). We can optionally choose a grid size of rows (`nx`) and columns (`ny`), which should be equal to the number of images you have. If it is less, then you can simply cut the `tsne` and `images` lists to be equal to `nx * ny`.

If you omit the `target=(nx, ny)` argument, RasterFairy will automatically choose an optimal grid size to be as square-shaped as possible. RasterFairy also has options for embedding them in a grid with irregular borders as well (see the GitHub page for more details).

You can also save the t-SNE points and their associated image paths for further processing in another environment.

In [12]:
tsne_path = "pca_features.json"

data = [{"path":os.path.abspath(img), "point":[float(x), float(y)]} for img, x, y in zip(images, tx, ty)]
with open(tsne_path, 'w') as outfile:
    json.dump(data, outfile)

print("saved t-SNE result to %s" % tsne_path)

saved t-SNE result to pca_features.json


In [10]:
import rasterfairy

# nx * ny = 1000, the number of images
nx = 40
ny = 25

# assign to grid
grid_assignment = rasterfairy.transformPointCloud2D(tsne, target=(nx, ny))
assignmentArray = np.array(grid_assignment[0])

In [11]:
print(len(assignmentArray))

1000


In [12]:
import csv
with open('theCSV.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    for row in range(0,len(assignmentArray)):
        num = assignmentArray[row]
        x = num[0]
        y = num[1]
        writer.writerow([x, y])

Now finally, we can make a new image of our grid. Set the `tile_width` and `tile_height` variables according to how big you want the individual tile images to be. The resolution of the output image is `tile_width * nx` x `tile_height * ny`. The script will automatically center-crop all the tiles to match the aspect ratio of `tile_width / tile_height`.

In [13]:
tile_width = 72
tile_height = 56

full_width = tile_width * nx
full_height = tile_height * ny
aspect_ratio = float(tile_width) / tile_height

grid_image = Image.new('RGB', (full_width, full_height))

for img, grid_pos in zip(images, grid_assignment[0]):
    idx_x, idx_y = grid_pos
    x, y = tile_width * idx_x, tile_height * idx_y
    tile = Image.open(img)
    tile_ar = float(tile.width) / tile.height  # center-crop the tile to match aspect_ratio
    if (tile_ar > aspect_ratio):
        margin = 0.5 * (tile.width - aspect_ratio * tile.height)
        tile = tile.crop((margin, 0, margin + aspect_ratio * tile.height, tile.height))
    else:
        margin = 0.5 * (tile.height - float(tile.width) / aspect_ratio)
        tile = tile.crop((0, margin, tile.width, margin + float(tile.width) / aspect_ratio))
    tile = tile.resize((tile_width, tile_height), Image.ANTIALIAS)
    grid_image.paste(tile, (int(x), int(y)))

matplotlib.pyplot.figure(figsize = (16,12))
imshow(grid_image)

FileNotFoundError: [Errno 2] No such file or directory: '../../Dataset/img_align_celeba/000997.jpg'

Finally, we can save the gridded t-SNE to disk as well.

In [18]:
grid_image.save("example-tSNE-grid.jpg")